### https://pennylane.ai/qml/demos/tutorial_plugins_hybrid.html

### BeamSplitter CV Operation on 2-qumode State

power of pennylane comes from plugin ecosystem
https://pennylane.ai/plugins.html

Strawberry Fields Fock backend allows for work on non-gaussian input states

In [1]:
import pennylane as qml
from pennylane import numpy as np

In [2]:
device_fock = qml.device("strawberryfields.fock", wires=2, cutoff_dim=2)

In [6]:
@qml.qnode(device_fock, diff_method="parameter-shift")
def photon_redirection(params):
    qml.FockState(1, wires=0)
    qml.Beamsplitter(params[0], params[1], wires=[0,1])
    return qml.expval(qml.NumberOperator(1))

a fock state / number state is an element of a Fock space with a well-defined number of particles/quanta. Here this is the state |1,0> that we are trying to optimise parameters in the BeamSplitter to maximise mean photon number in the second wire i.e. redirect the photon to wire 2 to achieve Fock state |0,1>

In [7]:
def cost(params):
    return -photon_redirection(params)

In [8]:
init_params = np.array([0.01, 0.01])
print(cost(init_params))

-9.999666671111081e-05


In [9]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)

steps = 100
params = init_params

for i in range(steps):
    params = opt.step(cost, params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

Cost after step     5: -0.0349558
Cost after step    10: -0.9969017
Cost after step    15: -1.0000000
Cost after step    20: -1.0000000
Cost after step    25: -1.0000000
Cost after step    30: -1.0000000
Cost after step    35: -1.0000000
Cost after step    40: -1.0000000
Cost after step    45: -1.0000000
Cost after step    50: -1.0000000
Cost after step    55: -1.0000000
Cost after step    60: -1.0000000
Cost after step    65: -1.0000000
Cost after step    70: -1.0000000
Cost after step    75: -1.0000000
Cost after step    80: -1.0000000
Cost after step    85: -1.0000000
Cost after step    90: -1.0000000
Cost after step    95: -1.0000000
Cost after step   100: -1.0000000
Optimized rotation angles: [1.57079633 0.01      ]


### Hybrid Computation

we can combine the performance of different qnodes, to work together in optimising a set of parameters. below parameters are optimised such that the output of either the qubit rotation qnode, or the photon redirection qnode, yields the same measurements.
here the parameters in photon redirection are tuned to give the same output as the random init values for qubit rotation

In [16]:
phi1, phi2 = 0.4, 0.2

In [15]:
dev_qubit = qml.device("default.qubit", wires=1)
dev_fock = qml.device("strawberryfields.fock", wires=2, cutoff_dim=10)


@qml.qnode(dev_qubit)
def qubit_rotation(phi1, phi2):
    """Qubit rotation QNode"""
    qml.RX(phi1, wires=0)
    qml.RY(phi2, wires=0)
    return qml.expval(qml.PauliZ(0))


@qml.qnode(dev_fock, diff_method="parameter-shift")
def photon_redirection(params):
    """The photon redirection QNode"""
    qml.FockState(1, wires=0)
    qml.Beamsplitter(params[0], params[1], wires=[0, 1])
    return qml.expval(qml.NumberOperator(1))


def squared_difference(x, y):
    """Classical node to compute the squared
    difference between two inputs"""
    return np.abs(x - y) ** 2

In [17]:
def cost(params, phi1=phi1, phi2=phi2):
    """Returns the squared difference between
    the photon-redirection and qubit-rotation QNodes, for
    fixed values of the qubit rotation angles phi1 and phi2"""
    qubit_result = qubit_rotation(phi1, phi2)
    photon_result = photon_redirection(params)
    return squared_difference(qubit_result, photon_result)

In [18]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)

steps = 100
params = np.array([0.01, 0.01])

for i in range(steps):
    params = opt.step(cost, params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.1912558
Cost after step    10:  0.0002023
Cost after step    15:  0.0000058
Cost after step    20:  0.0000002
Cost after step    25:  0.0000000
Cost after step    30:  0.0000000
Cost after step    35:  0.0000000
Cost after step    40:  0.0000000
Cost after step    45:  0.0000000
Cost after step    50:  0.0000000
Cost after step    55:  0.0000000
Cost after step    60:  0.0000000
Cost after step    65:  0.0000000
Cost after step    70:  0.0000000
Cost after step    75:  0.0000000
Cost after step    80:  0.0000000
Cost after step    85:  0.0000000
Cost after step    90:  0.0000000
Cost after step    95:  0.0000000
Cost after step   100:  0.0000000
Optimized rotation angles: [1.25357501 0.01      ]


In [19]:
result = [1.25357501, 0.01]
print(photon_redirection(result))
print(qubit_rotation(phi1, phi2))

0.9027010943137169
0.9027010963754598
